In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import random
import copy
import sys
import pygame
import random
from pygame import *
import numpy as np
from main import DinoGameEnv
import matplotlib.pyplot as plt
from agent import Agent
from model import Model
from collections import deque
%matplotlib inline

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
device

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


device(type='cpu')

In [2]:
os.chdir('/home/sans/Music/DinoAI')

In [3]:
env = DinoGameEnv()

In [6]:
env = DinoGameEnv()
states = env.reset()
env.allow_rendering = True
scores = 0
for i in range(10):
    action = np.random.choice(np.arange(0,4))
    S_, R, done = env.step(action)
    print(S_)
    scores += R
    
    if done:
        print('Max scores: ', scores)
        states = env.reset()
        scores = 0
        
    if i == 9:
        env.close()
        break

[  1.  71. 100.   0.   1.]
[  1.  44. 100.   0.   1.]
[  1.  43. 100.   0.   1.]
[  1.  42. 100.   0.   1.]
[ 1. 15. 91.  0.  1.]
[ 1. 14. 90.  0.  1.]
[ 1. 12. 89.  0.  0.]
[  1.  62. 100.   0.   0.]
[ 1. 34. 88.  0.  0.]
[ 1.  7. 61.  0.  0.]


In [19]:
agent = Agent(5, 3)

In [5]:
def dqn():
    scores_deque = deque(maxlen=100)
    scores = []
    n_episodes = 5000
    env.allow_rendering = True
    i = 1
    for episode in range(n_episodes):
        state = env.reset()
        done = False
        score = 0
        while not done:
            action = agent.act(state)
#             action = np.random.choice(np.arange(0,4))
            new_state, reward, done = env.step(action)
            agent.step(state, action, reward, new_state, done)
            state = new_state
            score += reward
            if done:
                break
                
        scores_deque.append(score)
        scores.append(score)
        
        print('\rEpisode {}\tAverage Score: {:.2f}\tReward: {} '.format(episode, np.mean(scores_deque), score), end="")
            
        if np.mean(scores_deque) > 50*i:
            print('Saving environment at scores {}'.format(np.mean(scores_deque)))
            torch.save(agent.local_nn.state_dict(), 'checkpoint' + '_' + str(50*i) + '.pth')
            
            i += 1
        if np.mean(scores_deque)>=1000.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode, np.mean(scores_deque)))
            torch.save(agent.local_nn.state_dict(), 'checkpoint.pth')
            break
            

In [6]:
dqn()

Episode 58	Average Score: 51.17	Reward: 708 Saving environment at scores 51.16949152542373
Episode 76	Average Score: 101.09	Reward: 338 Saving environment at scores 101.0909090909091
Episode 101	Average Score: 154.14	Reward: 651 Saving environment at scores 154.14
Episode 113	Average Score: 202.55	Reward: 272 Saving environment at scores 202.55
Episode 139	Average Score: 256.88	Reward: 645 Saving environment at scores 256.88
Episode 265	Average Score: 63.49	Reward: 45   

KeyboardInterrupt: 

In [20]:
agent.local_nn.load_state_dict(torch.load('checkpoint_250.pth'))

state = env.reset()
env.allow_rendering = True
score = 0
done = False
agent.local_nn.eval()
while not done:
    action = agent.act(state)
    new_state, reward, done = env.step(action)
#     agent.step(state, action, reward, new_state, done)
    state = new_state
    score += reward
    if done:
        print(score)
        break
        

-88
